In [1]:
import polars as pl

In [ ]:
#!wget https://krishna.gs.washington.edu/download/CADD-development/v1.7/training_data/GRCh38/simulation_SNVs.vcf.gz

--2024-11-23 22:25:33--  https://krishna.gs.washington.edu/download/CADD-development/v1.7/training_data/GRCh38/simulation_SNVs.vcf.gz
Resolving krishna.gs.washington.edu (krishna.gs.washington.edu)... 128.208.8.136
Connecting to krishna.gs.washington.edu (krishna.gs.washington.edu)|128.208.8.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57579931 (55M) [application/x-gzip]
Saving to: ‘simulation_SNVs.vcf.gz’

simulation_SNVs.vcf 100%[===================>]  54.91M  83.0MB/s    in 0.7s    

2024-11-23 22:25:35 (83.0 MB/s) - ‘simulation_SNVs.vcf.gz’ saved [57579931/57579931]



In [ ]:
#!wget https://krishna.gs.washington.edu/download/CADD-development/v1.7/training_data/GRCh38/humanDerived_SNVs.vcf.gz

--2024-11-23 22:29:46--  https://krishna.gs.washington.edu/download/CADD-development/v1.7/training_data/GRCh38/humanDerived_SNVs.vcf.gz
Resolving krishna.gs.washington.edu (krishna.gs.washington.edu)... 128.208.8.136
Connecting to krishna.gs.washington.edu (krishna.gs.washington.edu)|128.208.8.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56676936 (54M) [application/x-gzip]
Saving to: ‘humanDerived_SNVs.vcf.gz’

humanDerived_SNVs.v 100%[===================>]  54.05M  75.5MB/s    in 0.7s    

2024-11-23 22:29:48 (75.5 MB/s) - ‘humanDerived_SNVs.vcf.gz’ saved [56676936/56676936]



In [11]:
COORDINATES = ["chrom", "pos", "ref", "alt"]

In [9]:
def load_vcf(path):
    return (
        pl.read_csv(
            path,
            has_header=False,
            separator="\t",
            new_columns=["chrom", "pos", "id", "ref", "alt"],
            schema_overrides=dict(chrom=str),
        )
        .drop("id")
    )

cadd = pl.concat([
    load_vcf("simulation_SNVs.vcf.gz").with_columns(cadd_label=pl.lit(True)),
    load_vcf("humanDerived_SNVs.vcf.gz").with_columns(cadd_label=pl.lit(False)),
])
cadd

chrom,pos,ref,alt,cadd_label
str,i64,str,str,bool
"""1""",967602,"""C""","""G""",true
"""1""",967847,"""A""","""C""",true
"""1""",968093,"""G""","""A""",true
"""1""",968228,"""G""","""A""",true
"""1""",968299,"""G""","""A""",true
…,…,…,…,…
"""22""",50781424,"""C""","""G""",false
"""22""",50781566,"""G""","""A""",false
"""22""",50781718,"""C""","""T""",false


In [24]:
cadd.unique()

chrom,pos,ref,alt,cadd_label
str,i64,str,str,bool
"""6""",16167852,"""A""","""G""",true
"""11""",103780646,"""C""","""T""",true
"""9""",137658303,"""T""","""A""",false
"""9""",6530340,"""T""","""C""",false
"""5""",169827456,"""G""","""T""",true
…,…,…,…,…
"""4""",58931409,"""A""","""G""",false
"""1""",152899039,"""A""","""G""",true
"""8""",39490136,"""G""","""A""",true


In [ ]:
# CADD even has variants in both positive and negative set, what a mess
cadd.unique(COORDINATES)

chrom,pos,ref,alt,cadd_label
str,i64,str,str,bool
"""15""",60784012,"""G""","""A""",false
"""1""",21934711,"""T""","""A""",true
"""3""",143735149,"""A""","""G""",true
"""3""",18577145,"""C""","""T""",true
"""1""",36538814,"""T""","""G""",true
…,…,…,…,…
"""12""",102385225,"""G""","""A""",false
"""6""",122855134,"""C""","""T""",true
"""21""",31522863,"""T""","""C""",false


In [10]:
mendelian = pl.read_parquet("../../results/dataset/mendelian_traits_matched_9/test.parquet")
mendelian

chrom,pos,ref,alt,OMIM,consequence,label,tss_dist,match_group
str,i64,str,str,str,str,bool,i64,str
"""1""",1425822,"""C""","""G""",null,"""PLS""",false,48,"""PLS_4"""
"""1""",1615869,"""C""","""T""",null,"""PLS""",false,35,"""PLS_0"""
"""1""",1659060,"""G""","""A""",null,"""PLS""",false,47,"""PLS_4"""
"""1""",1659114,"""A""","""G""",null,"""PLS""",false,101,"""PLS_5"""
"""1""",2050958,"""T""","""C""",null,"""5_prime_UTR_variant""",false,149,"""5_prime_UTR_variant_7"""
…,…,…,…,…,…,…,…,…
"""X""",155613005,"""C""","""T""",null,"""PLS""",false,52,"""PLS_52"""
"""X""",155719093,"""C""","""A""",null,"""5_prime_UTR_variant""",false,4,"""5_prime_UTR_variant_101"""
"""X""",155881342,"""A""","""C""",null,"""PLS""",false,2,"""PLS_57"""


In [12]:
overlap_mendelian = mendelian.join(cadd, on=COORDINATES, how="inner")
overlap_mendelian

chrom,pos,ref,alt,OMIM,consequence,label,tss_dist,match_group,cadd_label
str,i64,str,str,str,str,bool,i64,str,bool
"""1""",5886890,"""T""","""C""",null,"""3_prime_UTR_variant""",false,104036,"""3_prime_UTR_variant_3""",true
"""1""",182671948,"""G""","""A""",null,"""PLS""",false,18,"""PLS_3""",true
"""10""",112375255,"""C""","""T""",null,"""5_prime_UTR_variant""",false,19,"""5_prime_UTR_variant_16""",true
"""10""",132331810,"""A""","""G""",null,"""5_prime_UTR_variant""",false,20,"""5_prime_UTR_variant_16""",true
"""11""",2390138,"""G""","""A""",null,"""5_prime_UTR_variant""",false,42,"""5_prime_UTR_variant_33""",true
…,…,…,…,…,…,…,…,…,…
"""11""",56289547,"""A""","""G""",null,"""3_prime_UTR_variant""",false,1542,"""3_prime_UTR_variant_9""",false
"""13""",43879803,"""C""","""T""",null,"""PLS""",false,62,"""PLS_17""",false
"""19""",6373528,"""A""","""G""",null,"""5_prime_UTR_variant""",false,56,"""5_prime_UTR_variant_75""",false


In [13]:
overlap_mendelian["label"].value_counts()

label,count
bool,u32
false,37


In [18]:
overlap_mendelian["match_group"].n_unique()

33

In [ ]:
# would 
overlap_mendelian.group_by(["label", "cadd_label"]).count()

/tmp/ipykernel_3910967/3574528169.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  overlap_mendelian.group_by(["label", "cadd_label"]).count()


label,cadd_label,count
bool,bool,u32
false,false,19
false,true,18


In [14]:
complex = pl.read_parquet("../../results/dataset/complex_traits_matched_9/test.parquet")
complex

chrom,pos,ref,alt,pip,trait,label,maf,ld_score,consequence,tss_dist,match_group
str,i64,str,str,f64,str,bool,f64,f64,str,i64,str
"""1""",867476,"""C""","""T""",0.00156,"""""",false,0.079465,44.053,"""non_coding_transcript_exon_var…",56446,"""non_coding_transcript_exon_var…"
"""1""",868052,"""T""","""C""",0.001791,"""""",false,0.077747,44.057,"""non_coding_transcript_exon_var…",55870,"""non_coding_transcript_exon_var…"
"""1""",868635,"""A""","""G""",0.004349,"""""",false,0.075255,43.639,"""non_coding_transcript_exon_var…",55287,"""non_coding_transcript_exon_var…"
"""1""",870176,"""T""","""A""",0.0,"""""",false,0.084371,37.271,"""non_coding_transcript_exon_var…",53746,"""non_coding_transcript_exon_var…"
"""1""",1052930,"""A""","""G""",0.001467,"""""",false,0.058385,46.907,"""non_coding_transcript_exon_var…",18823,"""non_coding_transcript_exon_var…"
…,…,…,…,…,…,…,…,…,…,…,…
"""22""",50368376,"""T""","""C""",0.0,"""""",false,0.19181,86.507,"""dELS""",3695,"""dELS_204"""
"""22""",50571623,"""C""","""T""",0.0,"""""",false,0.061159,23.9,"""dELS""",6291,"""dELS_202"""
"""22""",50671289,"""G""","""A""",0.0,"""""",false,0.036223,12.733,"""pELS_flank""",3125,"""pELS_flank_26"""


In [15]:
overlap_complex = complex.join(cadd, on=COORDINATES, how="inner")
overlap_complex

chrom,pos,ref,alt,pip,trait,label,maf,ld_score,consequence,tss_dist,match_group,cadd_label
str,i64,str,str,f64,str,bool,f64,f64,str,i64,str,bool
"""1""",6100638,"""A""","""G""",0.0,"""""",false,0.21997,48.887,"""3_prime_UTR_variant""",6133,"""3_prime_UTR_variant_1""",true
"""1""",8945251,"""C""","""T""",0.000528,"""""",false,0.01779,14.655,"""pELS""",615,"""pELS_1""",true
"""1""",17265255,"""C""","""T""",0.0,"""""",false,0.090727,65.903,"""dELS""",16156,"""dELS_14""",true
"""1""",23643640,"""C""","""T""",0.0,"""""",false,0.28617,40.919,"""dELS_flank""",48101,"""dELS_flank_0""",true
"""1""",50067414,"""T""","""C""",0.0,"""""",false,0.049883,45.136,"""non_coding_transcript_exon_var…",19399,"""non_coding_transcript_exon_var…",true
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20""",951868,"""C""","""T""",0.0,"""""",false,0.41336,19.908,"""dELS""",35533,"""dELS_182""",false
"""20""",1893313,"""G""","""A""",0.0,"""""",false,0.025223,15.178,"""pELS""",853,"""pELS_61""",false
"""20""",4225891,"""C""","""T""",0.0,"""""",false,0.0016623,8.7078,"""dELS""",23395,"""dELS_194""",false


In [16]:
overlap_complex["label"].value_counts()

label,count
bool,u32
true,9
false,134


In [17]:
overlap_complex["match_group"].n_unique()

130

In [20]:
overlap_complex.group_by(["label", "cadd_label"]).count()

/tmp/ipykernel_3910967/174212295.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  overlap_complex.group_by(["label", "cadd_label"]).count()


label,cadd_label,count
bool,bool,u32
true,false,1
true,true,8
false,false,55
false,true,79


In [26]:
# should get a p-value fisher exact test
# 1-sided

In [30]:
from scipy.stats import fisher_exact

fisher_exact([[1, 8], [55, 79]], alternative="two-sided").pvalue

0.0897618381919682

In [31]:
fisher_exact([[1, 8], [55, 79]], alternative="less").pvalue

0.07102197746064398

In [33]:
fisher_exact([[1, 8], [55, 79]], alternative="greater").pvalue

0.9903760956785749